In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm

In [21]:
# Load our data.
df = pd.read_csv("data/Normalized_Purchase_Card_Transactions.csv")
df_agencies = pd.read_csv("data/List_Agencies.csv")
df_vendors = pd.read_csv("data/List_Vendors.csv")

In [3]:
blank = np.zeros((df['AGENCY_NUM'].max() + 1, df['VENDOR_NAME_NUM'].max() + 1))

for index, row in tqdm(df.iterrows(), total = df.shape[0]):
    blank[row['AGENCY_NUM'], row['VENDOR_NAME_NUM']] += row['TRANSACTION_AMOUNT_NORM']

np.count_nonzero(blank)

  0%|          | 0/390109 [00:00<?, ?it/s]

44494

In [4]:
wordCounts = pd.DataFrame(blank)

bags = {i: df[df['AGENCY_NUM'] == i]['VENDOR_NAME_NUM'].unique() for i in df['AGENCY_NUM'].unique()}

In [5]:
def tf(counts):
    tfDF = pd.DataFrame()
    
    for index, row in counts.iterrows():
        tfDF[index] = row.div(row.sum())
        
    return tfDF

tf = pd.DataFrame(tf(wordCounts)).T

In [9]:
idf = tf.apply(lambda x: np.log((df['AGENCY_NUM'].max() + 1) / (np.count_nonzero(x.to_numpy())+1)), axis=0)

In [10]:
prod = tf*idf

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(prod).astype("float")
for i in range(len(similarity)):
    similarity[i][i] = 0

In [12]:
mostsimilar = np.argmax(similarity, axis=1)
mostsimilar_verbose = np.array([df_agencies.iloc[i][1] for i in mostsimilar])
mostsimilar_vendors = np.array([bags[i] for i in mostsimilar], dtype = object)

In [13]:
def summarize(agency = 0):
    """
    Gives brief information on the agency and its most similar agency.
    """
    agency_bag = bags[agency]
    agency_bag_verbose = np.array([df_vendors.iloc[i][1] for i in agency_bag])
    similar_agency_bag = bags[mostsimilar[agency]]
    similar_agency_bag_verbose = np.array([df_vendors.iloc[i][1] for i in similar_agency_bag])
    
    print("Selected Agency:", df_agencies.iloc[agency][1])
    print("Most Similar Agency:", mostsimilar_verbose[agency])
    print()
    print("Agency Vendors:", agency_bag_verbose)
    print()
    print("Most Similar Agency Vendors:", similar_agency_bag_verbose)

In [17]:
summarize(3)

Selected Agency: Board of Real Property Assessment & Appeals
Most Similar Agency: Commission of Judicial Disabilities & Tenure

Agency Vendors: ['STANDARD_OFFICE_SUPPLY' 'LASER_ART_INC' 'FEDEX_'
 'SUPRETECH_INCORPORATED' 'PAYPAL_CORPORATEEX' 'DUPONT_COMPUTER_INC'
 'METROPOLITAN_OFFICE_PR' 'CAPITAL_SERVICES_AND_S' 'BENJAMIN_OFFICE_SU'
 'PROBAR' 'SUPERIOR_COURIERS_LLC' 'DELTA_ASSOCIATES'
 'GRAYMAR_BUSINESS_SOLUT' 'UNITEDBUSINESSTECH' 'STAR_OFFICE_PRODUCTS_I'
 'DMI_DELL_KGOVT' 'DS_WATERS' 'HIGDON_INC' 'FLASH_GLASS__MIRROR_C'
 'DS_WATERS_STANDARD_COF' 'THOMSON_WESTTCD' 'HENDERSON_PROFESSIO'
 'MCKISSOCK_LP' 'APPRAISAL_INSTITUTE' 'US_FACILITIES_INC'
 'BANNER_STAFFING' 'SQ_KARMIC_KOLLECTIONS' 'COSTAR_GROUP_INC'
 'STAR_OFFICE_PRODUCTS' 'SQ_BWH_CONSULTANTS' 'GRTR_CPTL_ASSOC_REALTO'
 'DC_BAR' 'MICROFRAME' 'PITNEY_BOWES' 'WASHINGTON_BIZ_JOURNAL'
 'EDUCATION_TO_GO' 'PBILEASEDEQUIPMENT' 'SQ_SYLVIA_PARKER_E'
 'DIGITAL_COPIER' 'MB_STAFFING_SERVIC' 'COMCAST_OF_WASHINGTON'
 'DS_SERVICES_STANDARD_C' 'SQ

In [36]:
#df.set_index(['Unnamed: 0']).groupby(['AGENCY_NUM', 'VENDOR_NAME_NUM']).sum('TRANSACTION_AMOUNT').sort_values(['TRANSACTION_AMOUNT'], ascending=False)

df.groupby(['AGENCY_NUM', 'VENDOR_NAME_NUM'])['TRANSACTION_AMOUNT'].sum(axis=1).nlargest(2)




TypeError: sum() got an unexpected keyword argument 'axis'